In [1]:
from manim import *
from numpy import linalg as npl
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching cs_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s cs_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [100]:
class CoriolisCircle(Mobject):
    def __init__(self, center, radius, xy_range, xy_length, params, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.radius = radius
        self.xy_range = xy_range
        self.xy_length = xy_length

        self.omega = params[0]
        self.velocity = params[1]
        self.dot_init_y = params[2]

        # coordinate system c
        self.ax_c = Axes(x_range = self.xy_range, y_range = self.xy_range, x_length = self.xy_length, y_length = xy_length,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.center)
        self.origin = self.ax_c.c2p(0, 0, 0)
        self.ax_c_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x$", font_size = 36, color = BLACK))
        self.ax_c_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y$", font_size = 36, color = BLACK))
        self.add(self.ax_c, self.ax_c_xlabel, self.ax_c_ylabel)

        # coordinate system c'
        self.ax_c_star = Axes(x_range = self.xy_range, y_range = self.xy_range, x_length = self.xy_length, y_length = xy_length,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.center)
        self.ax_c_star_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x'$", font_size = 36, color = BLACK)).shift(0.5 * DOWN)
        self.ax_c_star_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y'$", font_size = 36, color = BLACK)).shift(0.5 * LEFT)
        self.ax_c_star_group = VGroup(self.ax_c_star, self.ax_c_star_xlabel, self.ax_c_star_ylabel)
        self.add(self.ax_c_star_group)

        # rotating circle
        self.circle = Circle(radius = self.radius, color = GREY, stroke_width = 2).move_to(self.origin)
        self.circle_group = VGroup(self.circle)
        n_ticks = 32
        for i in range(n_ticks):
            i_line = Line(start = self.origin + self.radius*UP + 0.05*UP, end = self.origin + self.radius*UP - 0.05*UP, color = GREY, stroke_width = 2).rotate(about_point = self.origin, angle = i/n_ticks*2*PI)
            self.circle_group.add(i_line)
        self.add(self.circle_group)

        # moving point
        self.point = Circle(radius = 0.1, color = RED, fill_opacity = 0.75).move_to(self.ax_c.c2p(self.xy_range[0], self.dot_init_y, 0.0))
        self.point.coordinate = np.array([self.xy_range[0], self.dot_init_y, 0.0])
        point_point = self.ax_c.c2p(*self.point.coordinate)
        self.point.coordinate_star = self.ax_c_star.p2c(point_point)
        self.add(self.point)


    # rotate c' and the reference circle
    def do_rotate(self, delta_t):
        self.ax_c_star_group.rotate(about_point = self.origin, angle = self.omega*delta_t/60)
        self.circle_group.rotate(about_point = self.origin, angle = self.omega*delta_t/60)


    # move point
    def do_move(self, delta_t):
        self.point.coordinate[0] += self.velocity*delta_t/60
        point_point = self.ax_c.c2p(*self.point.coordinate)
        self.point.coordinate_star = self.ax_c_star.p2c(point_point)
        self.point.move_to(self.ax_c.c2p(*self.point.coordinate))



class Coordinate(Mobject):
    def __init__(self, center, xy_range, xy_length, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.xy_range = xy_range
        self.xy_length = xy_length

        # coordinate system c
        self.ax_c = Axes(x_range = self.xy_range, y_range = self.xy_range, x_length = self.xy_length, y_length = xy_length,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.center)
        self.origin = self.ax_c.c2p(0, 0, 0)
        self.ax_c_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x$", font_size = 36, color = BLACK))
        self.ax_c_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y$", font_size = 36, color = BLACK))
        self.add(self.ax_c, self.ax_c_xlabel, self.ax_c_ylabel)


    # get dot
    def get_dot(self, coordinates):
        dot = Dot(radius = 0.025, color = RED).move_to(self.ax_c.c2p(*coordinates))
        return dot


class CoordinateStar(Mobject):
    def __init__(self, center, xy_range, xy_length, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.xy_range = xy_range
        self.xy_length = xy_length

        # coordinate system c'
        self.ax_c_star = Axes(x_range = self.xy_range, y_range = self.xy_range, x_length = self.xy_length, y_length = xy_length,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.center)
        self.ax_c_star_xlabel = self.ax_c_star.get_x_axis_label(Tex(r"$x'$", font_size = 36, color = BLACK))
        self.ax_c_star_ylabel = self.ax_c_star.get_y_axis_label(Tex(r"$y'$", font_size = 36, color = BLACK))
        self.ax_c_star_group = VGroup(self.ax_c_star, self.ax_c_star_xlabel, self.ax_c_star_ylabel)
        self.add(self.ax_c_star_group)


    # get dot
    def get_dot(self, coordinates):
        dot = Dot(radius = 0.025, color = RED).move_to(self.ax_c_star.c2p(*coordinates))
        return dot

In [107]:
%%capture_video --path "../animations/coriolis_simple/coriolis_simple_HD_offset_doublestop.mp4"
%%manim -qh --fps 60 $video_scene


class cs_Scene(ThreeDScene):
    def construct(self):
        self.camera.background_color = WHITE

        # physical parameters
        omega = 2*PI
        velocity = 4
        dot_init_y = -0.5                                       
        params = (omega, velocity, dot_init_y)

        # algorithm control parameters
        delta_t = 0.1
        dot_stop_t = 4.5
        dot_restart_t = 7

        dot_stop_t2 = 11.5
        dot_restart_t2 = 14

        # main object
        center = np.array([-4, 0, 0])
        radius = 2
        xy_range = [-2, 2, 1]
        xy_length = 5

        coriolis_circle = CoriolisCircle(center, radius, xy_range, xy_length, params)
        self.add(coriolis_circle)


        # static coordinate system
        c_center = np.array([4, 1.5, 0])
        c_xy_length = 2.5

        coordinate_system = Coordinate(c_center, xy_range, c_xy_length)
        self.add(coordinate_system)


        # static coordinate system
        c_star_center = np.array([4, -1.5, 0])
        c_star_xy_length = 2.5

        coordinate_star_system = CoordinateStar(c_star_center, xy_range, c_star_xy_length)
        self.add(coordinate_star_system)


        def coriolis_circle_updater(circle):
            t = t_tracker.get_value()
            circle.do_rotate(delta_t)

            if t > dot_stop_t and t < dot_restart_t or t > dot_stop_t2 and t < dot_restart_t2:
                circle.do_move(0)
            else:
                circle.do_move(delta_t)

            c_dot = coordinate_system.get_dot(coriolis_circle.point.coordinate)
            c_star_dot = coordinate_star_system.get_dot(coriolis_circle.point.coordinate_star)
            self.add(c_dot, c_star_dot)

        
        t_tracker = ValueTracker(0)
        self.wait(1)
        coriolis_circle.add_updater(coriolis_circle_updater)
        self.play(t_tracker.animate.set_value(15), rate_func= linear, run_time = 15)
        self.wait(2)

Output saved by creating file at ../animations/coriolis_simple/coriolis_simple_HD_offset_doublestop.mp4.
